In [1]:
import torch
from PIL import Image
import faiss
import numpy as np
import clip
import json
import glob

In [2]:
# Kiểm tra và sử dụng GPU nếu có, nếu không thì sử dụng CPU
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# Tải mô hình CLIP và preprocess
model, preprocess = clip.load("ViT-B/32", device=device)

In [ ]:
# Danh sách lưu trữ các vector đặc trưng và ánh xạ từ ID đến đường dẫn ảnh
features = []
id2img_fps = {}

# Lấy danh sách tất cả các file ảnh .jpg trong thư mục images và thư mục con của images
list_image = glob.glob("images/**/*.jpg", recursive=True)

# Duyệt qua từng hình ảnh và trích xuất vector đặc trưng
for id, image_path in enumerate(list_image):
    id2img_fps[id] = image_path  # Lưu ánh xạ từ ID đến đường dẫn hình ảnh
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)  # Chuẩn bị hình ảnh để đưa vào mô hình
    with torch.no_grad():
        image_features = model.encode_image(image).detach().cpu().numpy()  # Trích xuất vector đặc trưng từ hình ảnh
    features.append(image_features)  # Lưu vector đặc trưng vào danh sách

# Kết hợp tất cả các vector đặc trưng thành một mảng numpy
image_embeddings = np.concatenate(features)

# Lưu ánh xạ từ ID đến đường dẫn hình ảnh vào file image_path.json
with open('image_path.json', 'w') as f:
    f.write(json.dumps(id2img_fps))

In [ ]:
# Tạo FAISS index sử dụng cosine similarity (IndexFlatIP)
index = faiss.IndexFlatIP(image_embeddings.shape[1])  # IndexFlatIP phù hợp với cosine similarity
index.add(image_embeddings)  # Thêm các vector đặc trưng vào FAISS index

# Lưu FAISS index vào file bin
faiss.write_index(index, 'faiss_normal_ViT.bin')

print("FAISS index đã được tạo và lưu trữ thành công.")